In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import matplotlib.colors as cols
import matplotlib.cm as cm

#### the same as above but with eager execution enabled

In [1]:
import numpy as np
import tensorflow as tf
tf.enable_eager_execution()
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import matplotlib.colors as cols
import matplotlib.cm as cm

In [2]:
CODE_MSB_VALUE = 1<<3
def codeBoard(board):
    return np.array(list(map(
        lambda L:list(map(lambda l: [int(x) for x in bin((1<<l)+CODE_MSB_VALUE)[3:]], L)),
        board
    )))

In [3]:
def getRot3(board):
    rotations = [board]
    for i in range(3):
        rotations.append(np.rot90(rotations[-1]))
    return rotations

def augment(board):
    flip = np.transpose(board)
    return [codeBoard(v) for v in getRot3(board)+getRot3(flip)]

In [4]:
board = [[0,0,1,2],
         [1,1,2,0],
         [0,0,2,1],
         [1,2,0,0]
        ]

code = np.array(augment(board), dtype=np.float32)
segment_hight = code.shape[1]
segment_width = code.shape[2]
code_depth = code.shape[3]

In [11]:
l = getActionsIterator(board)
list(l)

[(0, 0), (0, 1), (1, 3), (2, 0), (2, 1), (3, 2), (3, 3)]

#### actions iterator

In [5]:
def getVacationsIterator(board):
    return zip(*np.nonzero(np.array(board) == 0))

orig_dirs = [(-1,0),(-1,-1),(0,-1),(1,-1)]
SingleDim = 4
MIN_LINE_SIZE = 3
isInRange = lambda loc: np.all(np.array(loc)>=0) and np.all(np.array(loc)<SingleDim)

def isStraitConnection(board, location, player):
    connection = []
    for shift in orig_dirs:
        line = []
        
        loc = location
        while True:
            loc = (loc[0]+shift[0], loc[1]+shift[1])
            if not isInRange(loc) or board[loc] != player: break
            line.append(loc)
        #print(line)
            
        loc = location
        while True:
            loc = (loc[0]-shift[0], loc[1]-shift[1])
            if not isInRange(loc) or board[loc] != player: break
            line.append(loc)
        #print(line)
        
        if len(line) >= MIN_LINE_SIZE-1:
            return True
    
    return False

def getOneMoveTransition(board, move):
    player, location = move
    is_terminal = isStraitConnection(board, location, player)
    next_board = board.copy()
    next_board[location] = player
    return (is_terminal, next_board)
    

#### test the environment

In [12]:
[(getOneMoveTransition(np.array(board),(2,loc))[0], loc) for loc in getVacationsIterator(board)]

[(False, (0, 0)),
 (False, (0, 1)),
 (True, (1, 3)),
 (False, (2, 0)),
 (True, (2, 1)),
 (True, (3, 2)),
 (False, (3, 3))]

#### test Conv2D

In [ ]:
k2D = np.array([[[2, 1, 0],[6, 0, 1],[2, 1, 0],[0,-1, 1]],
               [[2, 1, 3],[9,-7, 1],[2, 1,-9],[5, 0, 0]],
               [[0, 1, 3],[0, 0, 1],[2, 1,-3],[5,-1, 1]],
               [[2, 1,-3],[0,-2, 1],[2, 1, 3],[4, 0,11]]
              ], dtype=np.float32)
kernel2D = tf.reshape(k2D, k2D.shape+(1,), name='kernel2D')

#reshape for 2D convolution
code2D = code.reshape((1,code.shape[0]*code.shape[1],)+code.shape[2:])
print(tf.squeeze(tf.nn.conv2d(code2D, filters=kernel, strides=4, padding='VALID')).numpy())

#### test Conv1D

In [ ]:
k1D = np.array([[2, 1, 3],[9, 0, 1],
                [0,-1, 4],[5, 0,-3]], dtype=np.float32)
kernel1D = tf.reshape(k1D, k1D.shape+(1,), name='kernel1D')
#reshape for 1D convolution
code1D = code.reshape((1,code.shape[0]*code.shape[1]*code.shape[2],code.shape[3]))
print(tf.squeeze(tf.nn.conv1d(code1D, filters=kernel1D, stride=4, padding='VALID')).numpy())

#### build the model

In [ ]:
class vModel(tf.keras.Model):
    def __init__(self):
        super(vModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(units=44, name='dense1')
        self.dense2 = tf.keras.layers.Dense(units=1, name='dense2')
    
    def call(self, input):
        x = self.dense1(input)
        x = self.dense2(x)
        return tf.math.reduce_max(x, axis=0)

model = vModel()

#### get the gradient

In [ ]:
def grad(position, estimate):
    with tf.GradientTape() as t:
        y = model(position)
        loss = tf.square(y, estimate)
        grad = t.gradient(loss, model.trainable_variables)